In [4]:
import pandas as pd
import numpy as np
from os.path import join
import functions
import pickle
import re

### 메모

In [3]:
메모=pd.read_excel(r"C:\Users\DATA\Desktop\P1,P8법인실사\메모조회새창_20250704_1113.xlsx", dtype={"채무자키":str})

In [17]:
보증인 = pd.read_excel(r"C:\Users\DATA\Desktop\P1,P8법인실사\정상보증인목록.xlsx", dtype={"채무자키":str})

In [18]:
보증인_정리 = (
    보증인
    .drop_duplicates(subset=['채무자키', '보증인성명인'])  # 중복 제거
    .groupby('채무자키')['보증인성명인']
    .apply(list)
    .reset_index()
)

In [19]:
# 보증인 이름 최대 몇 명까지 있는지 계산
max_len = 보증인_정리['보증인성명인'].apply(len).max()

# 리스트를 여러 열로 분리
보증인_정리_split = 보증인_정리.copy()
for i in range(max_len):
    보증인_정리_split[f'보증인성명{i+1}'] = 보증인_정리_split['보증인성명인'].apply(lambda x: x[i] if i < len(x) else None)

# 원래 리스트 칼럼 제거
보증인_정리_split.drop(columns='보증인성명인', inplace=True)


In [21]:
보증인_정리_split.to_excel(r"C:\Users\DATA\Desktop\P1,P8법인실사\정상보증인_병렬.xlsx", index=False)

In [8]:
메모 = 메모.merge(보증인_정리_split, on='채무자키', how='left')
메모.head()

,채무자키,매각사구분,등록일자,등록유저,메모내용,메모구분,채무자명,전송결과,구분일,보증인성명1,보증인성명2,보증인성명3,보증인성명4,보증인성명5,보증인성명6,보증인성명7,보증인성명8
0,20412794,한울가람,★ 2025-06-13 ★,전병만,구분\t성명\t상태\t법조치 \t비고\n────────────────────...,요약,㈜미각,NaN,2025-06-13,김정희,None,None,None,None,None,None,None
1,20412760,한울가람,★ 2025-06-13 ★,전병만,구분\t성명\t상태\t법조치 \t비고\n────────────────────...,요약,㈜거산,NaN,2025-06-13,정수연,None,None,None,None,None,None,None
2,20412751,한울가람,★ 2025-06-13 ★,전병만,구분\t성명\t상태\t법조치 \t비고\n────────────────────...,요약,㈜건승,NaN,2025-06-13,신옥희,None,None,None,None,None,None,None
3,20411446,DNP-01,★ 2025-06-13 ★,전병만,구분\t성명\t상태\t법조치 \t비고\n────────────────────...,요약,㈜한양트레이딩,NaN,2025-06-13,박경옥,None,None,None,None,None,None,None
4,20429668,케이에스,★ 2024-08-01 ★,전병만,매각사 KS인데 한울가람으로 잘못등록된 거 정정\n\n양수도라인 \t하나은행...,DC,㈜아이텍스필,NaN,2024-08-01,정주병,None,None,None,None,None,None,None


In [12]:
불가문구 = r"(완납|면책|면탈|사망|이민|상실|패소|종결|제외)"

메모['불가문구여부'] = 메모.메모내용.str.extract(f'{불가문구}')

In [14]:
def 보증인포함여부(row):
    for col in ['보증인성명1', '보증인성명2', '보증인성명3', '보증인성명4', '보증인성명5', '보증인성명6', '보증인성명7','보증인성명8']:
        이름 = row[col]
        if pd.notnull(이름) and 이름 in row['메모내용']:
            return True
    return False

메모['보증인언급여부'] = 메모.apply(보증인포함여부, axis=1)


In [16]:
메모[메모.불가문구여부.notna() & 메모.보증인언급여부].to_excel(r"C:\Users\DATA\Desktop\P1,P8법인실사\불가문구메모.xlsx", index=False)

### 입금자구분

In [6]:
path = r"C:\Users\DATA\Desktop"
fn = "입금자구분"

# df = pd.read_excel(join(path, fn + ".xlsx"), dtype={"채무자키":str, "입금키":str})
# df.to_pickle(join(path, fn +".pkl"))
df_ori = pd.read_pickle(join(path, fn + ".pkl"))

In [10]:
# 성명로그
성명로그 = pd.read_excel(join(path, "성명로그_20250625_1310.xlsx"), dtype={"채무자키":str})

c:\Users\DATA\AppData\Local\anaconda3\envs\py_39\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [24]:
# 변경전/변경후 모두 합친 후, 채무자키별 고유값 집합 생성
성명로그_combined = 성명로그.melt(id_vars="채무자키", value_vars=["변경전", "변경후"], value_name="이름")
성명로그_combined["이름"] = 성명로그_combined["이름"].replace("test", pd.NA)

# 그룹별로 고유값 목록 만들기
grouped = 성명로그_combined.groupby("채무자키")["이름"].apply(lambda x: sorted(set(x.dropna()))).reset_index()

# 고유값을 컬럼으로 펼치기
성명이력 = grouped["이름"].apply(pd.Series)
성명이력.columns = [f"이름_{i+1}" for i in 성명이력.columns]
성명이력.insert(0, "채무자키", grouped["채무자키"])

성명이력

,채무자키,이름_1,이름_2,이름_3
0,20407819,유성일,유성일(로젠이사),NaN
1,20407820,배시영,배시영(금화통운),NaN
2,20407975,김하린,김하린(김세은),NaN
3,20408001,강나현,강보희,NaN
4,20408047,김나현,김나현(개명전:김보슬),NaN
...,...,...,...,...
857,20527133,김지안,김혜경,NaN
858,20527470,신건호,신건호(변경전 : 신현길),신현길
859,20527675,이상민,이재규,NaN
860,20527831,이윤지,이윤지(변경전 : 이향순),이향순


In [32]:
df = df_ori.copy()
df = df.drop_duplicates(subset="입금키", keep='first')
df = df[~(df.입금자구분=="주채무")]

print(df.columns)
print(성명이력.columns)

Index(['채무자키', '입금키', '입금자', '입금자구분', '입금구분', '입금메모', '입금비고', '채무자명', '개인법인',
       '보증인성명'],
      dtype='object')
Index(['채무자키', '이름_1', '이름_2', '이름_3'], dtype='object')


In [33]:
# df에 변경이름들 추가하기
print(len(df))
df = df.merge(성명이력, how='left')
print(len(df))

747972
747972


In [34]:
# 입금자 == 채무자
cols_to_check = ["채무자명", "이름_1", "이름_2", "이름_3"]

def is_match(row):
    depositor = str(row["입금자"])
    for col in cols_to_check:
        name = str(row.get(col, ""))
        if pd.isna(name):
            continue
        if depositor in name or name in depositor:
            return True
    return False

df["입금자가차주"] = df.apply(is_match, axis=1)

In [35]:
# 입금자 == 보증인
def match_guarantor(row):
    depositor = str(row["입금자"])
    guarantors = str(row.get("보증인성명", ""))
    
    if pd.isna(depositor) or pd.isna(guarantors):
        return False

    for name in guarantors.split(","):
        name = name.strip()
        if depositor in name or name in depositor:
            return True
    return False

df["입금자가보증인"] = df.apply(match_guarantor, axis=1)

In [45]:
# 법인채권인인데 입금자가 채무자
conds = [
    (df.개인법인=="법인") & df.입금자가차주, 
    (df.입금자구분=="채무자") & df.입금자가보증인,
    (df.입금자구분=="보증인") & df.입금자가차주,
    (~df.입금자가차주) & (~df.입금자가보증인)
]
values = [
    "법인이름으로입금된경우",
    "보증인",
    "채무자",
    "둘다아님"
]
df["확인필요건"] = np.select(conds, values, default=pd.NA)

In [50]:
확인필요건 = df[df.확인필요건.notna()].sort_values(by=["채무자키","입금자"])

In [54]:
functions.save_df_to_excel_underline(확인필요건, join(path, "입금자구분확인필요건.xlsx"), key_columns_no="채무자키")